# Eight Puzzle Solver

## Introduction

The **8-puzzle** is a sliding tile puzzle consisting of a 3x3 grid with eight numbered tiles and one blank space (*). The goal is to rearrange the tiles to match a given **goal state** by sliding the tiles into the blank space.

### What You Will Learn:
- How to represent the 8-puzzle as a **state space search problem**.
- How different search algorithms (uninformed and informed) solve the problem.
- How heuristics can improve search efficiency.
- How to compare different heuristics using experimental results.

### Example of an 8-Puzzle

1 2 3 4 5 6 7 8 *

# Eight Puzzle Solver

The 8-puzzle is a variation of the classic [15 puzzle](https://en.wikipedia.org/wiki/15_puzzle) with a 3x3 grid

This notebook lets you experiment with a simple implementation that provides three subclasses of the AIMA Problem:
 * **P8** : Algorithm A with **no heuristic**; provides a simple breadth first graph search
 * **P8_OOP** : Algoritm A with the heuristic of the number of **tiles out of place**
 * **P8_MHD** : Algoritm A with the heuristic of the **manhatten distance** for each tile to where it should be 

In [3]:
import sys
import search        # AIMA module for search problems
import random
import time



default_goal = '*12345678'
#default_goal = '1238*4765'

class P8(search.Problem):
    """A state is represented as a 9-character string with digits 1-8
    for tiles and '*' for a blank."""
    name = 'NIL'
    

    def __init__(self, goal=default_goal, initial=None, N=10):
        self.goal = goal
        self.initial = initial if initial else random_state(goal, N)

    def goal_test(self, s):
        """ Returns True iff s is a goal """
        return s == self.goal

    def actions(self, s):
        return actions8(s)

    def result(self, S, A):
        return result8(S,A)

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2
        from state1 via action, assuming cost c to get up to
        state1. The addional cost of every action is 1. """
        return c + 1

    def h(self, node):
        """Null heuristic for 8 puzzle: returns 0 for a goal 1 otherwise """
        return 0 if node == self.goal else 1
    

In [4]:
def actions8(S):
    """ Returns list of action possible in state S """
    action_table = {
        0:['down', 'right'],
        1:['down', 'left', 'right'],
        2:['down', 'left'],
        3:['up','down','right'],
        4:['up','down','left','right'],
        5:['up','down','left'],
        6:['up','right'],
        7:['up','left','right'],
        8:['up','left']}
    return action_table[S.index('*')]

def result8(S, A):
    """ Returns 8puzzle state after doing action A in state S"""
    blank = S.index('*')  # blank position, swap is location we want to movie it to
    if A == 'up':
        swap = blank - 3
        return ''.join([S[0:swap], '*', S[swap+1:blank], S[swap], S[blank+1:]])
    elif A == 'down':
        swap = blank + 3
        return ''.join([S[0:blank], S[swap], S[blank+1:swap], '*', S[swap+1:]])
    elif A == 'left':
        swap = blank - 1
        return ''.join([S[0:swap], '*', S[swap], S[blank+1:]])
    elif A == 'right':
        swap = blank + 1
        return ''.join([S[0:blank], S[swap], '*', S[swap+1:]])
    raise ValueError('Unrecognized action: ' + A)

In [6]:
def nodups_random_state(state, n):
    """Random walk of n steps without revisiting a state."""
    visited = set([state])
    for _ in range(n):
        actions = list(actions8(state))
        random.shuffle(actions)

        next_state = None
        for act in actions:
            candidate = result8(state, act)
            if candidate not in visited:
                next_state = candidate
                break

        # If stuck (all next states already visited), allow a move anyway
        if next_state is None:
            next_state = result8(state, random.choice(actions8(state)))

        visited.add(next_state)
        state = next_state

    return state


def printsoln(goal):
    """shows solution to 8 puzzle"""
    # path is list of Nodes from initial to goal
    path = goal.path()
    print(f"{len(path)-1} steps from {path[0].state} to {goal.state}")
    for node in path:
        a = node.action
        s = node.state
        print("%s\t%s\n\t%s\n\t%s\n" % (a,s[0:3],s[3:6],s[6:9]))

def print_state(s):
     print(f"{s[0:3]}\n{s[3:6]}\n{s[6:9]}\n")

def solve(initial=None, n=10):
    """Solves a random 8 puzzle problem and prints info"""
    if initial:
        s = initial
        print(f"P8: {initial} => {default_goal}")
    else:
        s = random_state(default_goal, n)
        print(f"\n{s} => {default_goal} ({n} steps from start)")        
        print(f"heur.\tsteps\tdepth\tstates\tsuccs\tEBF\tseconds")        
    
    for p in [P8(initial=s), P8_OOP(initial=s), P8_MHD(initial=s)]:
        solve1(p, n=n)

def solve1(p, n=10):
    ip = search.InstrumentedProblem(p)
    begin_time = time.time()
    solution = search.astar_search(ip)
    elapsed = time.time() - begin_time
    if solution:
        depth = len(solution.path())-1
        #eb = ebf(ip.succs, depth)
        print(f"{p.name}\t{n}\t{depth}\t{ip.states}\t{ip.succs}\t{elapsed:.5f}")
        #print(f"{p.name}\t{n}\t{depth}\t{ip.states}\t{ip.succs}\t{eb:.2f}\t{elapsed:.5f}")
        return solution
    else:
        print("  No solution found ☹")

def ebf(n, d):
    """ returns an estimate of the effective branching factor, which is the number
    of successors generated by a "typical" node for a given search problem.
      n: states whose successors have been generated.
      d: depth at which the solution node was found
      b: effective branching factor
      N = b + (b)**2 + ... + (b)**d
    There is no closed form solution to computing it, but this gives  a rough estimate"""
    
    return n ** (1/d)

            
def run_problems(steps=[5,10,15,18]):
    """ Solve an instance of the 8 puzzle from an inital state reached
    by a random walk of N steps for each N in the list steps"""
    for nsteps in steps:
        solve(n=nsteps)

We represent a state as a string of eight digits and a * for the blank.  Our **print_state** method prints the state in a 3x3 format.

We will use **goal_state** as the desired solution for our examples.

In [8]:
p8 = P8()
goal_state = "1234*5678"
print_state(goal_state)

123
4*5
678



To test our solver, we need to find a problem state that we know can be solved. Recall that a sliding tile puzzle is known to have two sets of states that are disjoint in that there is no way to get from a state in one set to a state in the other. 

Our random_state method takes a state and an integer and generates a goal state that is at most 10 steps away using a random walk.  There may be a way to get to it in fewer than 10 steps, but it will not require more.

In [10]:
problem_state = random_state(goal_state, 10)
print_state(problem_state)

273
415
68*

